In [6]:
import pandas as pd
import numpy as np
import os
import re
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from Tusc5IntegralUtils import *
from Tusc5ImageUtils import *

[2024-08-20 09:51:23,440][INFO]: Connecting Raphael@vfsmdatajoint01.fsm.northwestern.edu:3306


OperationalError: (2003, "Can't connect to MySQL server on 'vfsmdatajoint01.fsm.northwestern.edu' (timed out)")

# Importing Data

In [ ]:
# '''
# Grabs the fluoresence value from each cell from 'ImageAnalysis folder'

# >>> Need to change directory
# '''


# # Initial Data Import

# directory = '/Users/raphaeltinio/LabAnalysis/ImageAnalysis' # Needs to get to the /ImageAnalysis folder

# stack_names = [f for f in os.listdir(directory) if f != '.DS_Store']

# Cell_Results = pd.DataFrame(columns = ['File', 'Cell', 'Stain', 'X_vals', 'Y_vals'])

# for fname in stack_names:
#     result_directory = directory + "/" + fname + "/results"
    
#     csv_files = [f for f in os.listdir(result_directory) if f.endswith('.csv')] #incase there is a none .csv file for some reason

#     # Grabs number of cells in an image
#     ncells = np.unique([re.search(r'(\d+)(?=\.csv$)', s).group(1) if re.search(r'(\d+)(?=\.csv$)', s) else None for s in csv_files])

#     # Cell Counter
#     if len(Cell_Results) == 0:
#         n_add = 0

#     else:
#         n_add = Cell_Results.Cell.max() + 1 # counts the number of cells after an addition of a z-stack dataframe, so that it keeps counting up 

#     for n in ncells:
#         unique_cell = [f for f in csv_files if "cell" + n + '.csv' in f] # Grabs list of specific cell

#         cell_number = int(n) + n_add

#         for csv in unique_cell:

#             stain_number = csv.split('_')[0]
#             #stain_number = re.findall(r'\d+$', stain_number)[0]

#             single_csv = pd.read_csv(result_directory + '/' + csv, usecols = [1,2])

#             temp_frame = pd.DataFrame({
#                 'File': fname,
#                 'Cell': cell_number,
#                 'Stain': stain_number,
#                 'X_vals': [single_csv['X'].to_list()],
#                 'Y_vals': [single_csv['Y'].to_list()]})

#             Cell_Results = pd.concat([Cell_Results, temp_frame])


# Cell_Results = Cell_Results.reset_index(drop = True)
# Cell_Results['Cell'] = Cell_Results['Cell'].astype('int')
# Cell_Results = Cell_Results.sort_values('Cell')

# # Extracting the first four integers and the following letter into separate columns ({DJID}{Eye})
# Cell_Results['DJID'] = Cell_Results['File'].str.extract(r'(\d{4})').astype(int)
# Cell_Results['Eye'] = Cell_Results['File'].str.extract(r'(\d{4})([A-Za-z])')[1]

# #Renaming Glut1 to GluT1
# Cell_Results['Stain'] = Cell_Results['Stain'].apply(lambda x: 'GluT1' if x == 'Glut1' else x)

# Cell_Results.to_csv('Cell_Results.csv', index = False)

In [ ]:
'''
Accepts data from the new image analysis
'''

Cell_Results = pd.read_csv('old_batch_data.csv')
Cell_Results['X_vals'] = Cell_Results['X_vals'].apply(ast.literal_eval) # Makes it so cells containing lists are identifiable as lists by pandas
Cell_Results['Y_vals'] = Cell_Results['Y_vals'].apply(ast.literal_eval)
Cell_Results.head()

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip
0,"(3751, 4661, 5315, 6412, 7209, 8695, 10866, 14...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False
1,"(318, 343, 401, 395, 551, 679, 802, 1256, 1904...",1,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False
2,"(12172, 17385, 21508, 30542, 40600, 57843, 874...",1,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False
3,"(1263, 1966, 2227, 3352, 4363, 6310, 9002, 130...",1,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False
4,"(5187, 5846, 6538, 7720, 9219, 10671, 13400, 1...",5,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,True,49.773839,False


## DJID_GEN_Eye Merge

In [7]:
'''
'DJID_GEN_EYES.csv' contains DJIDs which connect DJIDs to genotype and which eye was in the experimental or control group

This code chunk merges that dataframe with Cell_Results


Takes into consideration if file_name has variations on eye types (e.g. RA, LD)
'''

Recorded_Info = pd.read_csv('DJID_GEN_EYES_DURATION.csv')
merged_df = pd.merge(Cell_Results, Recorded_Info[['DJID', 'Genotype', 'Experimental Eye', 'Control Eye', 'Time_Condition']], on='DJID', how='left')
merged_df.loc[merged_df['Eye'].str[0] == merged_df['Experimental Eye'].str[0], 'Group_Type'] = 'Experimental'
merged_df.loc[merged_df['Eye'].str[0] == merged_df['Control Eye'].str[0], 'Group_Type'] = 'Control'
Cell_Results = merged_df.drop(columns=['Experimental Eye', 'Control Eye'])

Cell_Results.loc[Cell_Results['Genotype'] == 'wildtype', 'eGFP_Value'] = False # Making sure wildtype retinas do not have false positive eGFP cells

Cell_Results.head()

NameError: name 'Cell_Results' is not defined

In [8]:
print('STATS BEFORE FILTER')

print('#'*50)
print('DJID:', Cell_Results.DJID.unique())
print('in_rip:', Cell_Results.in_rip.unique())
print('Group_type:', Cell_Results.Group_Type.unique())
print('Time Conditions:', Cell_Results.Time_Condition.unique())
print('Genotype:', Cell_Results.Genotype.unique())
print('#'*50)

print(Cell_Results.groupby('DJID')['Time_Condition'].count()/4)
print('#'*50)

STATS BEFORE FILTER
##################################################


NameError: name 'Cell_Results' is not defined

## Birth Data Merge


In [9]:
'''
Fetching a table to connect DJID to mouse age
Then merging that table to Cell_Results

'''

# DJ query and fetch
init_query = sln_animal.Animal.proj('dob', 'sex') * sln_animal.AnimalEvent * sln_animal.Deceased
birth_data = init_query.proj('dob', 'sex', 'date').fetch(format = 'frame')

# Reseting index and renaming columns
birth_data = birth_data.reset_index().drop(columns = ['event_id'])
birth_data = birth_data.rename(columns = {'animal_id': 'DJID', 'dob': 'birth', 'date': 'death'})

# Changing variables to date time vars
birth_data['birth'] = pd.to_datetime(birth_data['birth'])
birth_data['death'] = pd.to_datetime(birth_data['death'])

# Calculate the difference in days
birth_data['Age_Days'] = (birth_data['death'] - birth_data['birth']).dt.days

# Convert days to months (approximately, assuming ~30.44 days in a month)
birth_data['Age_Months'] = round(birth_data['Age_Days'] / 30.44,1)

# Merge
Cell_Results = Cell_Results.merge(birth_data, how = 'left', on = 'DJID')
Cell_Results = Cell_Results.drop(columns = ['birth', 'death', 'Age_Days'])
Cell_Results.head()

NameError: name 'sln_animal' is not defined

## Slice Separation and ID generation

In [10]:
'''
1) Adds a column to the dataframe which contains the z-stack slice seperation value

2) Adds a cell id

'''
Cell_Results['Slice_Seperation'] = Cell_Results['X_vals'].apply(lambda x: x[1]-x[0])

Cell_Results['Cell'] = Cell_Results.groupby(['file_name', 'mask_id']).ngroup()

NameError: name 'Cell_Results' is not defined

In [ ]:
Cell_Results.head()

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Time_Condition,Group_Type,sex,Age_Months,Slice_Seperation,Cell
0,"(3751, 4661, 5315, 6412, 7209, 8695, 10866, 14...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0
1,"(318, 343, 401, 395, 551, 679, 802, 1256, 1904...",1,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0
2,"(12172, 17385, 21508, 30542, 40600, 57843, 874...",1,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0
3,"(1263, 1966, 2227, 3352, 4363, 6310, 9002, 130...",1,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0
4,"(5187, 5846, 6538, 7720, 9219, 10671, 13400, 1...",5,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,True,49.773839,False,homo,15,Experimental,Male,4.6,0.15,1


In [5]:
plot_single_cell(Cell_Results.query('Cell == 120'), prominence= 20, distance = 10)

NameError: name 'plot_single_cell' is not defined

## Filters

In [10]:
'''
Goes through each cell in Cell_Results and calculates peaks

Prominence = 25 was chosen as it experimentally retained the most number of good cells.

'''

# Peak Filter

print(f'Original number of Cells: {len(Cell_Results)/4}')

remove_list = []

for cell_number in Cell_Results['Cell'].unique():
    
    CR_Query = Cell_Results.query('Cell == @cell_number').copy()

    slice_seperation = CR_Query.iloc[0]['Slice_Seperation']

    WGA_min_distance = int(2.25 / slice_seperation) # peaks must be 4 microns away from eachother
    DAPI_min_distance = int(10 / slice_seperation)
    
    index_max = len(CR_Query.iloc[0]['X_vals'])

    WGA_Y_vals = CR_Query.loc[CR_Query['Stain'] == 'WGA']['Y_vals'].iloc[0]
    DAPI_Y_vals = CR_Query.loc[CR_Query['Stain'] == 'DAPI']['Y_vals'].iloc[0]

    peaks_WGA, _ = find_peaks(WGA_Y_vals, prominence= 25, distance = WGA_min_distance)
    peaks_DAPI, _ = find_peaks(DAPI_Y_vals, prominence= 25, distance = DAPI_min_distance)

    '''
    WGA Filters

    '''
    
    # 1) Remove if less than two peaks
    if len(peaks_WGA) < 2:
        remove_list.append(cell_number)
        continue
    

    if peaks_WGA[1] != peaks_WGA[-1] and WGA_Y_vals[peaks_WGA[-1]] > WGA_Y_vals[peaks_WGA[1]]:
        remove_list.append(cell_number)
        continue

    # '''
    # DAPI Filters
    # '''

    if len(peaks_DAPI) < 1:
        remove_list.append(cell_number)
        continue

    elif len(peaks_DAPI) == 1 and peaks_DAPI[0] > peaks_WGA[0] and peaks_DAPI[0] < peaks_WGA[1]:
        pass

    if peaks_DAPI[0] != peaks_DAPI[-1] and DAPI_Y_vals[peaks_DAPI[-1]] > DAPI_Y_vals[peaks_DAPI[1]]:
        remove_list.append(cell_number)

# Removing undesirable cells from main list
remove_list = np.unique(remove_list)
Cell_Results = Cell_Results[~Cell_Results['Cell'].isin(remove_list)]

print(f'Remaining Cells after Filter: {len(Cell_Results)/4}')
print(f'Cell Numbers Removed {remove_list}, count : {len(remove_list)}')

Original number of Cells: 2580.0
Remaining Cells after Filter: 2084.0
Cell Numbers Removed [  28   33   43   44   52   72   81   85   96   98  100  101  104  109
  112  117  119  123  124  127  133  141  142  148  153  159  160  161
  162  163  164  165  166  167  170  171  174  177  180  182  183  185
  186  188  189  190  191  192  193  195  197  202  207  209  210  213
  214  215  216  218  219  220  221  222  223  225  226  227  228  229
  230  231  232  233  235  236  237  239  241  244  246  249  252  259
  260  264  267  271  272  275  278  282  283  284  289  290  291  292
  293  294  295  296  297  298  299  300  301  302  303  304  305  308
  311  314  316  318  322  326  327  332  354  361  416  432  441  449
  450  451  499  501  505  528  529  535  548  603  605  606  608  609
  619  620  633  642  643  657  658  659  666  672  674  679  680  681
  689  706  755  764  765  767  884  885  900  938  950  961  965  969
  973  974  997 1025 1026 1085 1088 1090 1138 1144 1177 1

In [11]:
print('STATS AFTER FILTER')

print('#'*50)
print('DJID:', Cell_Results.DJID.unique())
print('in_rip:', Cell_Results.in_rip.unique())
print('Group_type:', Cell_Results.Group_Type.unique())
print('Time Conditions:', Cell_Results.Time_Condition.unique())
print('Genotype:', Cell_Results.Genotype.unique())
print('#'*50)
print(Cell_Results.groupby('DJID')['Time_Condition'].count()/4)
print('#'*50)

STATS AFTER FILTER
##################################################
DJID: [3420 3527 3529 3574]
in_rip: [False  True]
Group_type: ['Experimental' 'Control']
Time Conditions: [15 60 90]
Genotype: ['homo' 'het' 'wildtype']
##################################################
DJID
3420    250.0
3527    838.0
3529    179.0
3574    817.0
Name: Time_Condition, dtype: float64
##################################################


## eGFP Finder (Retired)

In [12]:
# eGFP finder
eGFP_filtered = []
remove_from_eGFP = []

for cell_number in Cell_Results['Cell'].unique():

    CR_Query = Cell_Results.query('Cell == @cell_number').copy()

    slice_seperation = CR_Query.iloc[0]['Slice_Seperation']
    DAPI_min_distance = int(10 / slice_seperation)

    DAPI_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'DAPI']['Y_vals'].iloc[0], prominence= 25, distance = DAPI_min_distance)
    DAPI_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'DAPI', 'Y_vals'].iloc[0])[DAPI_idx[0]]

    # eGFP peak needs to be at least half of DAPI peak to be considered eGFP positive
    eGFP_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'eGFP']['Y_vals'].iloc[0], prominence= 100, distance = 10, height = int(DAPI_peak_val/2))
    eGFP_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'eGFP', 'Y_vals'].iloc[0])[eGFP_idx]

    WGA_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'WGA']['Y_vals'].iloc[0], prominence= 25)
    WGA_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'WGA', 'Y_vals'].iloc[0])[WGA_idx]

    if len(eGFP_peak_val) == 1:

        # If eGFP peak is not within WGA peaks, cell is not considered eGFP positive
        if eGFP_idx < WGA_idx[0] or eGFP_idx > WGA_idx[1]:
            remove_from_eGFP.append(cell_number)
        
        # If eGFP is not at least 1/2 the size of 
        if eGFP_peak_val > WGA_peak_val[1]/2:
            eGFP_filtered.append(cell_number)


eGFP_filtered = list(set(eGFP_filtered))
eGFP_filtered = sorted([i for i in eGFP_filtered if i not in remove_from_eGFP])

print(f'Cells with eGFP: {eGFP_filtered}, count: {len(eGFP_filtered)}')

# Labeling Cells T5 Positive or Negative
Cell_Results['T5'] = 'Negative'
Cell_Results.loc[Cell_Results['Cell'].isin(eGFP_filtered), 'T5'] = 'Positive'

Cells with eGFP: [1, 14, 26, 35, 46, 51, 65, 75, 103, 145, 172, 179, 181, 217, 265, 274, 307, 331, 336, 337, 342, 369, 877, 878, 882, 889, 893, 902, 909, 910, 912, 913, 919, 920, 921, 937, 946, 954, 956, 959, 960, 964, 1129, 1157, 1161, 1173, 1175, 1606, 1610, 1631, 1633, 1637, 1640, 1651, 1702, 1714, 1719, 1726, 1747, 1748, 1756, 1759, 1764, 1768, 1770, 1772, 1773, 1789, 1801, 1810, 1811, 1814, 1823, 1825, 1844, 1853, 1854, 1859, 1869, 1874, 1881, 1882, 1883, 1898, 1905, 1911, 1912, 1937, 1943, 1948, 1950, 1954, 1956, 1963, 1976, 2002, 2021, 2022, 2027, 2028, 2087, 2114, 2126, 2131, 2138, 2149, 2151, 2236, 2252, 2276, 2277, 2286, 2289, 2294, 2301, 2317, 2320, 2328, 2330, 2331, 2339, 2349, 2363, 2369, 2371, 2398, 2413, 2432, 2453, 2468, 2469, 2470, 2488, 2494, 2542, 2550, 2579], count: 137


In [13]:
# Comparison between
positive_old_identifier = np.array(eGFP_filtered)
positive_new_identifier = Cell_Results.loc[Cell_Results['eGFP_Value'] == True].Cell.unique()
print('Number of cells in old eGFP finder', len(positive_old_identifier))
print('Number of cells in new eGFP finder', len(positive_new_identifier))
print('Percent of old cells from old eGFP finder in new eGFP finder:', np.isin(positive_old_identifier, positive_new_identifier).sum()/ len(np.isin(positive_old_identifier, positive_new_identifier)) * 100, '%')



Number of cells in old eGFP finder 137
Number of cells in new eGFP finder 217
Percent of old cells from old eGFP finder in new eGFP finder: 61.31386861313869 %


In [14]:
Cell_Results

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Time_Condition,Group_Type,sex,Age_Months,Slice_Seperation,Cell,T5
0,"(3751, 4661, 5315, 6412, 7209, 8695, 10866, 14...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
1,"(318, 343, 401, 395, 551, 679, 802, 1256, 1904...",1,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
2,"(12172, 17385, 21508, 30542, 40600, 57843, 874...",1,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
3,"(1263, 1966, 2227, 3352, 4363, 6310, 9002, 130...",1,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
4,"(5187, 5846, 6538, 7720, 9219, 10671, 13400, 1...",5,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,True,49.773839,False,homo,15,Experimental,Male,4.6,0.15,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,"(4442, 5633, 7399, 9474, 11693, 15637, 20282, ...",140,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,False,6.515115,False,homo,90,Experimental,Male,1.4,0.15,2572,Negative
10316,"(23108, 24246, 25996, 28027, 29579, 32583, 355...",150,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive
10317,"(11052, 11853, 13073, 14637, 15954, 18866, 217...",150,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive
10318,"(15706, 17205, 19480, 22970, 26698, 32407, 364...",150,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive


## Summary of Cell_Results

In [15]:
'''
Sanity Check


'''

print('Genotype', Cell_Results['Genotype'].unique())
print('T5 Labelling', Cell_Results['Stain'].unique())
print('eGFP Types', Cell_Results['eGFP_Value'].unique())

print('\n')
print('###'*12)
print('\n')

'''
Summary of mouse information
'''

def extract_before_underscore(input_string):
    parts = input_string.split("_", 1)
    if len(parts) > 1:
        return parts[0]
    
eyes = Cell_Results['file_name'].apply(extract_before_underscore).unique()

def extract_first_four_integers(input_string):
    # Find all substrings of consecutive digits
    integers = re.findall(r'\d+', input_string)
    
    # Convert the found substrings into integers
    integers = [int(i) for i in integers]
    
    # Return the first four integers
    return integers[:4][0]

animals = list(map(extract_first_four_integers, eyes))
animals = pd.Series(animals).unique() # Extracting the unqiue values

left = 0
right = 0

for i in eyes:
    if 'R' in i:
        right += 1
    if 'L' in i:
        left +=1

positive_count = Cell_Results['eGFP_Value'].value_counts()[True]/4
total = (len(Cell_Results))/4


###
print(f'Number of mice: {len(animals)}')
print(f'Mice DJIDs: {animals}')
print(f'Number of eyes: {len(eyes)}, Left = {left} R = {right}')

###
print('\n')
print('###'*12)
print('\n')

###
print('Number of Cells', Cell_Results['Cell'].count()/4)
print('eGFP Positive', (Cell_Results.groupby('eGFP_Value').size()/4)[True])
print('eGFP Negative', (Cell_Results.groupby('eGFP_Value').size()/4)[False])
print('Positive over Total Percentage', positive_count/total*100)

Genotype ['homo' 'het' 'wildtype']
T5 Labelling ['DAPI' 'eGFP' 'WGA' 'GluT1']
eGFP Types [False  True]


####################################


Number of mice: 4
Mice DJIDs: [3420 3527 3529 3574]
Number of eyes: 11, Left = 5 R = 6


####################################


Number of Cells 2084.0
eGFP Positive 217.0
eGFP Negative 1867.0
Positive over Total Percentage 10.412667946257198


In [16]:
Cell_Results.columns

Index(['Y_vals', 'mask_id', 'Stain', 'X_vals', 'file_name', 'DJID', 'Eye',
       'eGFP_Value', 'eGFP_Raw_Intensity', 'in_rip', 'Genotype',
       'Time_Condition', 'Group_Type', 'sex', 'Age_Months', 'Slice_Seperation',
       'Cell', 'T5'],
      dtype='object')

In [17]:
Cell_Results.groupby(['Genotype','DJID','sex'])['Age_Months'].unique()

Genotype  DJID  sex   
het       3527  Female    [2.2]
homo      3420  Male      [4.6]
          3574  Male      [1.4]
wildtype  3529  Female    [2.2]
Name: Age_Months, dtype: object

# Calculations

## Defining Functions

In [25]:
'''
Function: Calculating WGA Peaks

Creates columns: WGA_Middle_Indices, Length (um)

'''

def WGA_Peaks_Finder(dataframe, prom_val:int = 25):

    # Function to find WGA peaks for each cell (via .groupby)
    def find_peaks_in_cell(cell_df):
        # Filter rows where Stain is 'WGA'
        df_WGA = cell_df.loc[cell_df['Stain'] == 'WGA']
        
        slice_seperation = df_WGA.iloc[0]['Slice_Seperation']
        WGA_min_distance = int(2.25 / slice_seperation) 
        WGA_peak_indicies = find_peaks(df_WGA['Y_vals'].iloc[0], prominence = prom_val, distance = WGA_min_distance)[0]


        return WGA_peak_indicies

    # Apply the peak finder function across cells grouped by 'Cell'
    grouped = dataframe.groupby('Cell')
    slice_separation = grouped['Slice_Seperation'].first().iloc[0]

    grouped_WGA_indices = grouped.apply(find_peaks_in_cell).reset_index().rename(columns={0: 'WGA_Middle_Indices'})    # Reset index to flatten the DataFrame and rename



    grouped_WGA_indices['Length'] = grouped_WGA_indices['WGA_Middle_Indices'].apply(lambda x: (x[1] - x[0]) * slice_separation)

    #Merge the peak indices back into the original DataFrame
    result_df = pd.merge(dataframe, grouped_WGA_indices, on='Cell', how='left')

    return result_df


In [26]:
Cell_Results

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Time_Condition,Group_Type,sex,Age_Months,Slice_Seperation,Cell,T5
0,"(3751, 4661, 5315, 6412, 7209, 8695, 10866, 14...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
1,"(318, 343, 401, 395, 551, 679, 802, 1256, 1904...",1,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
2,"(12172, 17385, 21508, 30542, 40600, 57843, 874...",1,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
3,"(1263, 1966, 2227, 3352, 4363, 6310, 9002, 130...",1,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,False,0.696460,False,homo,15,Experimental,Male,4.6,0.15,0,Negative
4,"(5187, 5846, 6538, 7720, 9219, 10671, 13400, 1...",5,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3420L_GLUT1_647_WGA_594_0001,3420,L,True,49.773839,False,homo,15,Experimental,Male,4.6,0.15,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,"(4442, 5633, 7399, 9474, 11693, 15637, 20282, ...",140,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,False,6.515115,False,homo,90,Experimental,Male,1.4,0.15,2572,Negative
10316,"(23108, 24246, 25996, 28027, 29579, 32583, 355...",150,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive
10317,"(11052, 11853, 13073, 14637, 15954, 18866, 217...",150,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive
10318,"(15706, 17205, 19480, 22970, 26698, 32407, 364...",150,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3574RB_light_WGA594_GLUT1_647_TUSC5_003,3574,RB,True,18.000000,False,homo,90,Experimental,Male,1.4,0.15,2579,Positive


In [27]:
'''
Function: Calculates "Top" and "Bottom" INDICES to be used for integral boundaries

Creates columns: WGA_Top_Indices, WGA_Bottom_Indices
'''

def Top_Bottom_Indices(dataframe, microns_extension: float = 1.5):
    # Grouping / Getting Slice Separation / Calculating peaks
    grouped = dataframe.groupby('Cell')
    slice_separation = grouped['Slice_Seperation'].first()
    max_idx = Cell_Results.groupby('Cell')['X_vals'].first().apply(lambda x: len(x))
    first_peaks = grouped['WGA_Middle_Indices'].apply(lambda x: x.iloc[0] if len(x) > 0 else [np.nan, np.nan])

    # Calculate index offset for each cell based on the slice separation
    index_offset = (microns_extension / slice_separation).fillna(0).astype(int)

    # Separate the peak indices
    l_middle = first_peaks.apply(lambda x: x[0] if len(x) > 0 else np.nan)
    r_middle = first_peaks.apply(lambda x: x[1] if len(x) > 1 else np.nan)

    # Calculate modified indices
    l_top = l_middle - index_offset
    r_bot = r_middle + index_offset

    # Prepare to merge corrections for indices exceeding boundaries
    max_idx = grouped['X_vals'].apply(lambda x: len(x.iloc[0]))

    # Adjust indices to ensure they do not exceed the size of the array
    l_top = np.maximum(l_middle - index_offset, 0)
    r_bot = np.minimum(r_bot, max_idx)

    # Creating DataFrame for merging
    idx_df = pd.DataFrame({
        'Cell': grouped.size().index,  # Assures alignment with group keys
        'WGA_Top_Indices': list(zip(l_top, l_middle)),
        'WGA_Bottom_Indices': list(zip(r_middle, r_bot))
    })

    # Merge adjusted index data back into the original DataFrame
    result_df = pd.merge(dataframe, idx_df, on='Cell', how='left')

    return result_df

In [28]:
'''
Function: Calculates "Top", "Middle", "Bottom" INTEGRALS

Creates columns: Middle_Integral, Top_Integral, Bottom_Integral
'''

def TopMidBot_Integrals(dataframe):
    

    # Allows you to calculate integrals through .apply()
    def integral_calculator(arr1, arr2, integral_section:str = 'Middle'):

        if integral_section == 'Middle':

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]]

        if integral_section == 'Top':

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]] 

        if integral_section == 'Bottom': 

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]]
        return peak_yvals


    for section in ['Middle', 'Top', 'Bottom']:
        dataframe[f'{section}_Integral'] = dataframe.apply(lambda x: np.sum(integral_calculator(x['Y_vals'], x[f'WGA_{section}_Indices'], integral_section = section)), axis =1)
    
    return dataframe

In [29]:
'''
Function: Calculates "Top", "Bottom" INTEGRAL RATIOS

# Calculating the ratio of Glut1 over WGA (normalization) .5 microns about the WGA peaks

Creates columns: [WGA or GluT1]_[Top or Bot]_Surface_Integral
'''

def Surface_Integrals(dataframe):
    # Initialize a list to store results dataframes for each group
    results = []

    # Group by 'Cell' and process each group
    for (cell, group) in dataframe.groupby('Cell'):
        # Filter for WGA and GluT1 stains
        df_WGA = group.loc[group['Stain'] == 'WGA']
        df_GluT1 = group.loc[group['Stain'] == 'GluT1']


        peak_indices = df_WGA['WGA_Middle_Indices'].iloc[0]
        x_vals = df_WGA['X_vals'].iloc[0]

        # Compute slice separation and radius offset
        slice_separation = group['Slice_Seperation'].iloc[0]
        radius = 0.5
        idx_offset = int(radius / slice_separation)

        # Define borders based on peak indices and radius offset
        top_lborder = max(peak_indices[0] - idx_offset, 0)
        top_rborder = min(peak_indices[0] + idx_offset, len(x_vals))
        bottom_lborder = max(peak_indices[1] - idx_offset, 0)
        bottom_rborder = min(peak_indices[1] + idx_offset, len(x_vals))

        # Calculate integrals for 'GluT1' and 'WGA'
        top_surface_integral_GluT1 = np.sum(df_GluT1.iloc[0]['Y_vals'][top_lborder:top_rborder])
        bottom_surface_integral_GluT1 = np.sum(df_GluT1.iloc[0]['Y_vals'][bottom_lborder:bottom_rborder]) 
        top_surface_integral_WGA = np.sum(df_WGA.iloc[0]['Y_vals'][top_lborder:top_rborder])
        bottom_surface_integral_WGA = np.sum(df_WGA.iloc[0]['Y_vals'][bottom_lborder:bottom_rborder+1])

        # Ratio calculations
        Top_G_over_W = top_surface_integral_GluT1 / top_surface_integral_WGA
        Bottom_G_over_W = bottom_surface_integral_GluT1 / bottom_surface_integral_WGA

        # Append results as a DataFrame to list
        results.append(pd.DataFrame({
            'Cell': [cell],
            'GluT1_Top_Surface_Integral': [top_surface_integral_GluT1],
            'GluT1_Bot_Surface_Integral': [bottom_surface_integral_GluT1],
            'WGA_Top_Surface_Integral': [top_surface_integral_WGA],
            'WGA_Bot_Surface_Integral': [bottom_surface_integral_WGA],
            'Top_Surface_Ratio': [Top_G_over_W],
            'Bot_Surface_Ratio': [Bottom_G_over_W]
        }))

    # Concatenate all results into a single DataFrame
    results_df = pd.concat(results, ignore_index=True)

    # Merge adjusted index data back into the original DataFrame

    dataframe = dataframe.merge(results_df, on='Cell', how='left')
    
    return dataframe

In [30]:
"""
Function: Reshapes columns, so that each group has a column with '{Stain}_{Integral_Type}'. 4 x 3 = 12 columns are added
"""

def Reshape_Integrals(dataframe):

    # Make pivot table
    pivot_df = dataframe.pivot_table(
        index=['Cell'], 
        columns='Stain',
        values=['Middle_Integral', 'Top_Integral', 'Bottom_Integral'],
        aggfunc='first' 
    )

    # Flatten the multi-level column headers and format them
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns.values]

    pivot_df.reset_index(inplace=True) # Cell is a column used to merge 

    result_df = pd.merge(dataframe, pivot_df)

    return result_df

In [31]:
Cell_Results_Calcs = Cell_Results.copy()

# These functions find indices
Cell_Results_Calcs = WGA_Peaks_Finder(Cell_Results_Calcs)
Cell_Results_Calcs = Top_Bottom_Indices(Cell_Results_Calcs)

# These functions calculate integrals using the index boundaries
Cell_Results_Calcs = TopMidBot_Integrals(Cell_Results_Calcs)
Cell_Results_Calcs = Reshape_Integrals(Cell_Results_Calcs)
Cell_Results_Calcs = Surface_Integrals(Cell_Results_Calcs)

## Export

In [32]:
'''
Making new dataframe for export
- Keeps certain columns
- Renames columns
- Mapping column categories to new names (e.g. eGFP Positive -> 1)
'''

# Columns to Keep
keep_cols = ['DJID', 'Genotype', 'Eye', 'sex', 'Age_Months', 'eGFP_Value', 'Group_Type', 'in_rip', 'Length', 'Time_Condition',
             'GluT1_Top_Surface_Integral', 'GluT1_Bot_Surface_Integral', 'WGA_Top_Surface_Integral', 'WGA_Bot_Surface_Integral',
             'GluT1_Top_Integral', 'GluT1_Middle_Integral', 'GluT1_Bottom_Integral',
             'WGA_Top_Integral', 'WGA_Middle_Integral', 'WGA_Bottom_Integral']

# Columns to rename
renamed_cols = {'sex': 'Sex',
                'Length': 'Length_um',
                'in_rip': 'In_Rip',
                'Group_Type' :'Experimental_Condition'}

Export_df = Cell_Results_Calcs.groupby('Cell').first().loc[:, keep_cols].reset_index(drop = True)
Export_df = Export_df.rename(columns = renamed_cols)

# Remapping values
Export_df['eGFP_Value'] = Export_df['eGFP_Value'].astype(int)
Export_df['In_Rip'] = Export_df['In_Rip'].astype(int)

Exp_Con_map = {'Experimental': 'Light Flicker', 'Control': 'Dark'}
Export_df['Experimental_Condition'] = Export_df['Experimental_Condition'].map(Exp_Con_map)#.fillna(Export_df['Experimental_Condition'])

# Checking for NA vals
if Export_df.isna().sum().sum() > 0:
    print('There are NA values, check for problems')

Export_df

,DJID,Genotype,Eye,Sex,Age_Months,eGFP_Value,Experimental_Condition,In_Rip,Length_um,Time_Condition,GluT1_Top_Surface_Integral,GluT1_Bot_Surface_Integral,WGA_Top_Surface_Integral,WGA_Bot_Surface_Integral,GluT1_Top_Integral,GluT1_Middle_Integral,GluT1_Bottom_Integral,WGA_Top_Integral,WGA_Middle_Integral,WGA_Bottom_Integral
0,3420,homo,L,Male,4.6,0,Light Flicker,0,2.55,15,107922,170093,1069585,653832,76333,367993,157541,779221,1528910,856820
1,3420,homo,L,Male,4.6,1,Light Flicker,0,2.55,15,489732,354335,1548185,1153001,249080,1501794,317760,969538,3064160,1372434
2,3420,homo,L,Male,4.6,0,Light Flicker,0,3.45,15,383891,114257,1362099,596600,198085,844785,133175,874124,2177701,806164
3,3420,homo,L,Male,4.6,0,Light Flicker,0,2.40,15,252370,373467,2158679,1376614,154632,958229,332392,1592229,3209467,1677220
4,3420,homo,L,Male,4.6,0,Light Flicker,0,2.25,15,795411,604414,2374917,1593088,346857,2008454,654046,1193593,3984508,2072681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,3574,homo,RB,Male,1.4,0,Light Flicker,0,2.25,90,173203,103086,204985,137970,84298,510634,76409,167956,385244,186094
2080,3574,homo,RB,Male,1.4,0,Light Flicker,0,2.55,90,198939,89867,183307,126046,95331,500859,81478,143140,381719,158889
2081,3574,homo,RB,Male,1.4,0,Light Flicker,0,3.45,90,186540,24240,131795,66246,96137,397021,11024,98053,337108,32823
2082,3574,homo,RB,Male,1.4,0,Light Flicker,0,2.25,90,192895,100214,277712,178721,102712,408723,93310,249679,504981,221298


In [41]:
Export_df = Export_df.loc[Export_df['Genotype'] != 'wildtype']
#Export_df.to_csv('old_batch_processed.csv', index = False)